In [83]:
import pandas as pd
import numpy as np
import math
import re
import time

In [84]:
def combine_categories_set(row):
    result = row.category_1.lower()
    
    if row.category_2:
        result = result + " " + row.category_2.lower()
    if row.category_3:
        result = result + " " + row.category_3.lower()
    if row.category_4:
        result = result + " " + row.category_4.lower()
    if row.category_5:
        result = result + " " + row.category_5.lower()
    if row.category_6:
        result = result + " " + row.category_6.lower()
    if row.category_7:
        result = result + " " + row.category_7.lower()
    
    result = set(re.split(r"[\>|\&| ]+", result))
    return result

def combine_categories(row):
    result = row.category_1.lower()
    
    if row.category_2:
        result = result + " > " + row.category_2.lower()
    if row.category_3:
        result = result + " > " + row.category_3.lower()
    if row.category_4:
        result = result + " > " + row.category_4.lower()
    if row.category_5:
        result = result + " > " + row.category_5.lower()
    if row.category_6:
        result = result + " > " + row.category_6.lower()
    if row.category_7:
        result = result + " > " + row.category_7.lower()

    return result

In [85]:
# https://www.google.com/basepages/producttype/taxonomy.en-US.txt
cat = pd.read_csv("./data/taxonomy.csv", encoding="ISO-8859-1")
cat.fillna(False, inplace=True)
cat["combined"] = cat.apply(lambda x: combine_categories(x), axis=1)
cat["combined_set"] = cat.apply(lambda x: combine_categories_set(x), axis=1)
print(cat.shape)
cat.head(2)

(5427, 9)


,category_1,category_2,category_3,category_4,category_5,category_6,category_7,combined,combined_set
0,Animals & Pet Supplies,False,False,False,False,False,False,animals & pet supplies,"{animals, pet, supplies}"
1,Animals & Pet Supplies,Live Animals,False,False,False,False,False,animals & pet supplies > live animals,"{animals, live, pet, supplies}"


In [86]:
df = pd.read_csv("./data/emails.csv", encoding="ISO-8859-1")
print(df.shape)
df.head(2)

(100, 2)


,subject,full_text
0,Prana Patterns on Point + a FREE $20,PLUS the $20 Almost Gift Card To view this ema...
1,Now 25% OFF every doggone thing. Everything. D...,The September Style sale The September Style s...


In [87]:
# def contains_word(s, w):
#     return (' ' + w + ' ') in (' ' + s + ' ')

# subjects = df.iloc[0:10].subject.apply(lambda x: x.lower())

# start_time = time.time()
# categories = []

# for subject in subjects:
#     category = set()
#     for i, row in cat.iterrows():
#         words = row["combined_set"]
#         for w in words:
#             if contains_word(subject, w):
#                 category.add(i)
#     categories.append(list(category))

# print("--- %s seconds ---" % (time.time() - start_time))
# # --- 3.7804958820343018 seconds ---
# # df.categories = categories

In [88]:
subjects = df.subject.apply(lambda x: x.lower())

start_time = time.time()
categories = []

for subject in subjects:
    category = set()
    subject = set(re.split(r"[\>|\&| ]+", subject))

    for i, row in cat.iterrows():
        words = row["combined_set"]
        
        if len(words.intersection(subject))>0: 
            category.add(i)

    categories.append(list(category))

print("--- %s seconds ---" % (time.time() - start_time))
# --- 3.6054348945617676 seconds ---
df["category_ids"] = categories

--- 38.86279606819153 seconds ---


In [132]:
# def print_categories(list_of_ids):
#     for i in list_of_ids:
#         category = cat.iloc[i, :-1]
#         print(" > ".join([i for i in category if i != False]))
        
# print_categories(list(category))

Toys & Games > Toys > Riding Toys > Rocking & Spring Riding Toys
Electronics > Audio > Audio Components > Headphones & Headsets
Toys & Games > Toys > Activity Toys > Coiled Spring Toys


In [104]:
def convert_categories_index_to_text(categories):
    result = []
    for i in categories:
        result.append(cat.iloc[i]["combined"])
        result.append("*****")
    return result

df["categories"] = df.apply(lambda x: convert_categories_index_to_text(x["category_ids"]), axis=1)

In [105]:
# QA

position = 0

print(df.iloc[position]["subject"])
print(df.iloc[position]["categories"])


Prana Patterns on Point + a FREE $20
['sporting goods > exercise & fitness > weight lifting > free weight accessories > weight bars', '*****', 'sporting goods > exercise & fitness > weight lifting > free weights', '*****', 'food, beverages & tobacco > food items > fruits & vegetables > fresh & frozen vegetables > greens > on choy', '*****', 'sporting goods > exercise & fitness > weight lifting > free weight accessories', '*****', 'sporting goods > exercise & fitness > weight lifting > free weight accessories > free weight storage racks', '*****', 'sporting goods > exercise & fitness > weight lifting > free weight accessories > weight bar collars', '*****', 'arts & entertainment > hobbies & creative arts > arts & crafts > crafting patterns & molds', '*****', 'arts & entertainment > hobbies & creative arts > arts & crafts > crafting patterns & molds > beading patterns', '*****', 'arts & entertainment > hobbies & creative arts > arts & crafts > crafting patterns & molds > craft molds', '*

In [12]:
# next steps
# get this to run faster
# clean up categories and assess results
# how do we handle singular v.s. plural?
# should we remove stop words?
# what categories should we add? (shoes example)
# using more than just subject (e.g. full_text)
# using google vision to understand the email better (image recognition, OCR, etc)
### use this info to add new categories
# explore using different weights (would this make sense for us?)